In [ ]:
import itertools
import os
import sys
from itertools import combinations_with_replacement
from math import e
import gc

import numpy as np
import pandas as pd
from numba import njit
from plotly.subplots import make_subplots
from vectorbt.indicators import nb
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import vectorbt as vbt
from lib.utils import file_to_data_frame, LR, ExtendedPortfolio, plot_series_vs_scatters, dropnaninf, get_best_index

In [ ]:
file = "/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/shorts/Binance_BTCUSDT_minute_3000.csv"
_, ohlcv = file_to_data_frame(file)
ohlcv. head()

In [ ]:
close = ohlcv["Close"]
volume = ohlcv["Volume"]
lr_ind = LR.run(close)
print(lr_ind.lr.shape)

In [ ]:
lag = list(range(7,13))
ma_ind = vbt.MA.run(lr_ind.lr, lag, short_name="lr_ma")
mstd_ind = vbt.MSTD.run(lr_ind.lr, lag)

In [ ]:
@njit
def combination_nb(ma, std, thld):
    return ma + thld * std

LR_MULTIPLIER = vbt.IndicatorFactory(
    input_names=['ma', 'std'],
    param_names=['thld'],
    output_names=['mu']
).from_apply_func(combination_nb, use_ray=True)
t = np.linspace(0,2,10, endpoint=True)
ups = LR_MULTIPLIER.run(ma_ind.ma, mstd_ind.mstd, thld=t, short_name="ups_mu")
pitfalls = LR_MULTIPLIER.run(ma_ind.ma, mstd_ind.mstd, thld=-t, short_name="pitfalls_mu")
pitfalls.mu.head()

In [ ]:
lr_entries = lr_ind.lr_below(pitfalls.mu)
lr_exits = lr_ind.lr_above(ups.mu)
lr_entries.head()

In [ ]:
vol_ma = vbt.MA.run(volume, lag)
vol_ma.ma.head()

In [ ]:
@njit
def multiplier_nb(values, m):
    return m * values

MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['m'],
    output_names=['mu']
).from_apply_func(multiplier_nb)

In [ ]:
# Volume part:
vol_threshold = np.linspace(0, 2, 10)
vol_multiplier = MULTIPLIER.run(vol_ma.ma, vol_threshold, short_name="vol_mu")
vol_entries = vol_multiplier.mu_below(volume)
del vol_multiplier, volume, vol_ma
print("sum", vol_entries.sum(axis=0))
print("shape", vol_entries.shape)

In [ ]:
gc.collect()
lr_entries.columns = lr_entries.columns.rename("lag", level=-1)
vol_entries.columns = vol_entries.columns.rename("lag", level=-1)
final_entries = lr_entries & vol_entries
del lr_entries, vol_entries


In [ ]:
# optimizamos lag, lr_thrld y vol_thrld buscando el máximo mean en el espacio entre entry signals
entries_test_arr = pd.Series((True,  False, False, False, False, False, False, False, True, True, False)).vbt.signals.first()
exits_test_arr =   pd.Series((False, False, False, True,  False, False, True,  True, False, False, True)).vbt.signals.first()

In [ ]:
_final_entries = pd.DataFrame(np.where(final_entries == True, 1, np.nan), index=final_entries.index, columns=final_entries.columns)
_final_entries

In [ ]:
_all_signals =  pd.DataFrame(np.where(lr_exits == True, -1, _final_entries), index=final_entries.index, columns=final_entries.columns)
_all_signals

In [ ]:
@njit
def k_mean(col, arr, *args):
    indexes = np.where(np.isfinite(arr))[0]
    lr = args[0]
    n = 1
    adder = 0
    counter = 0
    while n < len(indexes):
        i = indexes[n]
        prev = indexes[n-1]
        if arr[prev] == 1 and arr[i] == -1:
            adder += np.mean(lr[prev+1:i +1])
            counter += 1
        n +=1
    return adder / counter
_all_signals.vbt.reduce(k_mean, lr_ind.lr.to_numpy())

In [ ]:
@njit
def double_multiplier_nb(values, x, y):
    return values*x, values*y

MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['x', 'y'],
    output_names=['x_mu','y_mu']
).from_apply_func(double_multiplier_nb)
x = np.linspace(0,2,5, endpoint=True)
y = -np.linspace(0,2,5, endpoint=True)
tp_sl = MULTIPLIER.run(mstd_ind.mstd, x, y, param_product=True, short_name="tp_sl")

In [ ]:
tp_exits = lr_ind.lr_above(tp_sl.x_mu)
sl_exits = lr_ind.lr_below(tp_sl.y_mu)
final_exits = tp_exits | sl_exits
final_exits.columns = final_exits.columns.rename("lag", level=-1)
final_exits

In [ ]:
portfolio_kwargs = dict(
    direction='longonly',
    freq='m',
)
port = ExtendedPortfolio.from_signals(close, entries=final_entries, exits=final_exits, **portfolio_kwargs, max_logs=0)

In [ ]:
elr = port.expected_log_returns()
top_elr = elr.nlargest(25, keep="all")

In [ ]:
sharpe = dropnaninf(port.sharpe_ratio())
top_sharpe = sharpe.nlargest(25, keep="all")

In [ ]:
top_elr.index.intersection(top_sharpe.index)

In [ ]:
port.stats(column=top_elr.index[0])

In [ ]:
port.stats(column=top_sharpe.index[0])

In [ ]:
final_entries_plot = (final_entries.where(final_entries == True, np.nan)).vbt.scatterplot()
lr_plot = plot_series_vs_scatters([lr_ind.lr, pitfalls.mu], [lr_entries, lr_exits])
vol_plot = plot_series_vs_scatters([vol_multiplier.mu, volume], [vol_entries])

In [ ]:
def add_all_subplots(fig, row, col, list):
    for a in list:
        fig.add_trace(a, row=row, col=col)

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02)
add_all_subplots(fig, 1, 1, lr_plot.data)
add_all_subplots(fig, 2, 1, vol_plot.data)
add_all_subplots(fig, 3, 1, final_entries_plot.data)
fig.update_layout(height=700, legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()